22BAI1173\
04/02/2025\
Lab 5\
L39+L40\
chatgpt: https://chatgpt.com/share/67a1ff4e-8e5c-8004-9277-a7837a0cda5a

# Question 1 - Comparative Study of edge detection performance on Synthetic vs. Natural Images.
Analyse the result using edge metrics.
  a. Edge Strength (Gradient Magnitude)\
  b. Edge Sharpness (Variance of Laplacian)\
  c. Edge Density\
  d. Edge Contrast (Michelson Contrast)


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
natural_original_gray = cv2.imread('natural.png', cv2.IMREAD_UNCHANGED)
natural_gray = cv2.imread('natural.png', cv2.IMREAD_GRAYSCALE)

In [ ]:
synthetic_original_gray = cv2.imread('synthetic.png', cv2.IMREAD_UNCHANGED)
synthetic_gray = cv2.imread('synthetic.png', cv2.IMREAD_GRAYSCALE)

synthetic_original_gray = cv2.GaussianBlur(synthetic_original_gray, (5, 5), 0)
synthetic_gray = cv2.GaussianBlur(synthetic_gray, (5, 5), 0)

noise = np.random.normal(0, 25, synthetic_gray.shape).astype(np.uint8)
noise_3_channel = cv2.cvtColor(noise, cv2.COLOR_GRAY2BGR)

synthetic_original_gray = cv2.add(synthetic_original_gray, noise_3_channel)
synthetic_gray = cv2.add(synthetic_gray, noise)

In [ ]:
def apply_sobel(image):
    sobel_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
    return cv2.magnitude(sobel_x, sobel_y)

def apply_prewitt(image):
    prewitt_kernel_x = np.array([[1, 0, -1], [1, 0, -1], [1, 0, -1]], dtype=np.float64)
    prewitt_kernel_y = np.array([[1, 1, 1], [0, 0, 0], [-1, -1, -1]], dtype=np.float64)
    prewitt_x = cv2.filter2D(image, cv2.CV_64F, prewitt_kernel_x)
    prewitt_y = cv2.filter2D(image, cv2.CV_64F, prewitt_kernel_y)
    return cv2.magnitude(prewitt_x, prewitt_y)

def apply_roberts(image):
    roberts_kernel_x = np.array([[1, 0], [0, -1]], dtype=np.float64)
    roberts_kernel_y = np.array([[0, 1], [-1, 0]], dtype=np.float64)
    roberts_x = cv2.filter2D(image, cv2.CV_64F, roberts_kernel_x)
    roberts_y = cv2.filter2D(image, cv2.CV_64F, roberts_kernel_y)
    return cv2.magnitude(roberts_x, roberts_y)

In [ ]:
def edge_strength(edges):
    return np.mean(edges)

def edge_sharpness(image):
    laplacian = cv2.Laplacian(image, cv2.CV_64F)
    return np.var(laplacian)

def edge_density(edges, threshold=50):
    return np.sum(edges > threshold) / edges.size

def edge_contrast(edges):
    return (np.max(edges) - np.min(edges)) / (np.max(edges) + np.min(edges))

In [ ]:
images = {
    'Synthetic Original': synthetic_original_gray,
    'Synthetic Processed': synthetic_gray,
    'Natural Original': natural_original_gray,
    'Natural Processed': natural_gray
}

In [ ]:
results = []
processed_images = {}
for img_name, img in images.items():
    sobel_edges = apply_sobel(img)
    prewitt_edges = apply_prewitt(img)
    roberts_edges = apply_roberts(img)

    # Normalize edge images for visualization
    def normalize(image):
        return cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

    processed_images[f'{img_name} - Sobel'] = normalize(sobel_edges)
    processed_images[f'{img_name} - Prewitt'] = normalize(prewitt_edges)
    processed_images[f'{img_name} - Roberts'] = normalize(roberts_edges)

    results.append({
        'Image Type': img_name,
        'Operator': 'Sobel',
        'Edge Strength': edge_strength(sobel_edges),
        'Edge Sharpness': edge_sharpness(img),
        'Edge Density': edge_density(sobel_edges),
        'Edge Contrast': edge_contrast(sobel_edges)
    })

    results.append({
        'Image Type': img_name,
        'Operator': 'Prewitt',
        'Edge Strength': edge_strength(prewitt_edges),
        'Edge Sharpness': edge_sharpness(img),
        'Edge Density': edge_density(prewitt_edges),
        'Edge Contrast': edge_contrast(prewitt_edges)
    })

    results.append({
        'Image Type': img_name,
        'Operator': 'Roberts',
        'Edge Strength': edge_strength(roberts_edges),
        'Edge Sharpness': edge_sharpness(img),
        'Edge Density': edge_density(roberts_edges),
        'Edge Contrast': edge_contrast(roberts_edges)
    })

In [ ]:
df = pd.DataFrame(results)
df

,Image Type,Operator,Edge Strength,Edge Sharpness,Edge Density,Edge Contrast
0,Synthetic Original,Sobel,269.230559,84998.447776,0.932743,1.0
1,Synthetic Original,Prewitt,190.600698,84998.447776,0.901129,1.0
2,Synthetic Original,Roberts,107.206034,84998.447776,0.726655,1.0
3,Synthetic Processed,Sobel,269.231469,84998.715744,0.932743,1.0
4,Synthetic Processed,Prewitt,190.601345,84998.715744,0.901132,1.0
5,Synthetic Processed,Roberts,107.206400,84998.715744,0.726658,1.0
6,Natural Original,Sobel,115.711676,1921.869272,0.568398,1.0
7,Natural Original,Prewitt,83.006885,1921.869272,0.501835,1.0
8,Natural Original,Roberts,26.774081,1921.869272,0.186481,1.0
9,Natural Processed,Sobel,115.726193,1921.996979,0.568369,1.0


In [ ]:
fig, axes = plt.subplots(len(images), 4, figsize=(16, 16))

for i, (img_name, img) in enumerate(images.items()):
    sobel_edges = processed_images[f'{img_name} - Sobel']
    prewitt_edges = processed_images[f'{img_name} - Prewitt']
    roberts_edges = processed_images[f'{img_name} - Roberts']

    axes[i, 0].imshow(img, cmap='gray')
    axes[i, 0].set_title(f'{img_name} - Original')
    axes[i, 0].axis('off')

    axes[i, 1].imshow(sobel_edges, cmap='gray')
    axes[i, 1].set_title(f'{img_name} - Sobel')
    axes[i, 1].axis('off')

    axes[i, 2].imshow(prewitt_edges, cmap='gray')
    axes[i, 2].set_title(f'{img_name} - Prewitt')
    axes[i, 2].axis('off')

    axes[i, 3].imshow(roberts_edges, cmap='gray')
    axes[i, 3].set_title(f'{img_name} - Roberts')
    axes[i, 3].axis('off')

plt.tight_layout()
plt.show()

Output hidden; open in https://colab.research.google.com to view.

# Question 2 - Fusion of Multiple Edge Detectors
  a. Weighted average Fusion\
  b. Max/Min Fusion\
  c. Logical Operations (Binary Fusion)- OR , AND

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
natural_original_gray = cv2.imread('natural.png', cv2.IMREAD_UNCHANGED)
natural_gray = cv2.imread('natural.png', cv2.IMREAD_GRAYSCALE)

In [ ]:
synthetic_original_gray = cv2.imread('synthetic.png', cv2.IMREAD_UNCHANGED)
synthetic_gray = cv2.imread('synthetic.png', cv2.IMREAD_GRAYSCALE)

synthetic_original_gray = cv2.GaussianBlur(synthetic_original_gray, (5, 5), 0)
synthetic_gray = cv2.GaussianBlur(synthetic_gray, (5, 5), 0)

noise = np.random.normal(0, 25, synthetic_gray.shape).astype(np.uint8)
noise_3_channel = cv2.cvtColor(noise, cv2.COLOR_GRAY2BGR)

synthetic_original_gray = cv2.add(synthetic_original_gray, noise_3_channel)
synthetic_gray = cv2.add(synthetic_gray, noise)

In [ ]:
def apply_sobel(image):
    sobel_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
    return cv2.magnitude(sobel_x, sobel_y)

def apply_prewitt(image):
    prewitt_kernel_x = np.array([[1, 0, -1], [1, 0, -1], [1, 0, -1]], dtype=np.float64)
    prewitt_kernel_y = np.array([[1, 1, 1], [0, 0, 0], [-1, -1, -1]], dtype=np.float64)
    prewitt_x = cv2.filter2D(image, cv2.CV_64F, prewitt_kernel_x)
    prewitt_y = cv2.filter2D(image, cv2.CV_64F, prewitt_kernel_y)
    return cv2.magnitude(prewitt_x, prewitt_y)

def apply_roberts(image):
    roberts_kernel_x = np.array([[1, 0], [0, -1]], dtype=np.float64)
    roberts_kernel_y = np.array([[0, 1], [-1, 0]], dtype=np.float64)
    roberts_x = cv2.filter2D(image, cv2.CV_64F, roberts_kernel_x)
    roberts_y = cv2.filter2D(image, cv2.CV_64F, roberts_kernel_y)
    return cv2.magnitude(roberts_x, roberts_y)

In [ ]:
def weighted_average_fusion(edges, weights=[0.33, 0.33, 0.34]):
    return sum(w * e for w, e in zip(weights, edges))

def max_fusion(edges):
    return np.maximum.reduce(edges)

def min_fusion(edges):
    return np.minimum.reduce(edges)

def logical_or_fusion(edges):
    return np.bitwise_or.reduce([(e > 50).astype(np.uint8) * 255 for e in edges])

def logical_and_fusion(edges):
    return np.bitwise_and.reduce([(e > 50).astype(np.uint8) * 255 for e in edges])

In [ ]:
def compute_edge_metrics(image):
    edge_strength = np.mean(image)
    edge_sharpness = cv2.Laplacian(image, cv2.CV_64F).var()
    edge_density = np.count_nonzero(image) / image.size
    edge_contrast = (np.max(image) - np.min(image)) / (np.max(image) + np.min(image) + 1e-6)
    return edge_strength, edge_sharpness, edge_density, edge_contrast

In [ ]:
def normalize(image):
    return cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

In [ ]:
images = {
    'Synthetic Original': synthetic_original_gray,
    'Synthetic Processed': synthetic_gray,
    'Natural Original': natural_original_gray,
    'Natural Processed': natural_gray
}

In [ ]:
results = {}
metrics = []
processed_images = {}
for img_name, img in images.items():
    sobel_edges = apply_sobel(img)
    prewitt_edges = apply_prewitt(img)
    roberts_edges = apply_roberts(img)

    processed_images[f'{img_name} - Sobel'] = normalize(sobel_edges)
    processed_images[f'{img_name} - Prewitt'] = normalize(prewitt_edges)
    processed_images[f'{img_name} - Roberts'] = normalize(roberts_edges)

    fusion_methods = {
        'Weighted Fusion': weighted_average_fusion([sobel_edges, prewitt_edges, roberts_edges]),
        'Max Fusion': max_fusion([sobel_edges, prewitt_edges, roberts_edges]),
        'Min Fusion': min_fusion([sobel_edges, prewitt_edges, roberts_edges]),
        'Logical OR Fusion': logical_or_fusion([sobel_edges, prewitt_edges, roberts_edges]),
        'Logical AND Fusion': logical_and_fusion([sobel_edges, prewitt_edges, roberts_edges])
    }

    results[img_name] = fusion_methods
    for fusion_name, fusion_img in fusion_methods.items():
        metrics.append([img_name, fusion_name, *compute_edge_metrics(fusion_img)])

In [ ]:
columns = ['Image', 'Fusion Method', 'Edge Strength', 'Edge Sharpness', 'Edge Density', 'Edge Contrast']
edge_metrics_df = pd.DataFrame(metrics, columns=columns)
edge_metrics_df

,Image,Fusion Method,Edge Strength,Edge Sharpness,Edge Density,Edge Contrast
0,Synthetic Original,Weighted Fusion,188.207245,102195.883002,0.998575,1.0
1,Synthetic Original,Max Fusion,278.560928,254479.972017,0.998575,1.0
2,Synthetic Original,Min Fusion,92.308881,76833.614948,0.942644,1.0
3,Synthetic Original,Logical OR Fusion,251.079470,17842.233939,0.984625,1.0
4,Synthetic Original,Logical AND Fusion,164.458874,281390.757057,0.644937,1.0
5,Synthetic Processed,Weighted Fusion,188.207863,102196.202819,0.998575,1.0
6,Synthetic Processed,Max Fusion,278.561830,254480.668429,0.998575,1.0
7,Synthetic Processed,Min Fusion,92.309218,76833.877467,0.942643,1.0
8,Synthetic Processed,Logical OR Fusion,251.079836,17840.344654,0.984627,1.0
9,Synthetic Processed,Logical AND Fusion,164.459606,281390.547136,0.644940,1.0


In [ ]:
num_fusion_methods = len(next(iter(results.values())))
fig, axes = plt.subplots(len(images), num_fusion_methods + 1, figsize=(24, 16))

for i, (img_name, fusion_methods) in enumerate(results.items()):
    axes[i, 0].imshow(normalize(images[img_name]), cmap='gray', vmin=0, vmax=255)
    axes[i, 0].set_title(f'{img_name} - Original')
    axes[i, 0].axis('off')

    for j, (fusion_name, fusion_img) in enumerate(fusion_methods.items(), start=1):
        axes[i, j].imshow(normalize(fusion_img), cmap='gray', vmin=0, vmax=255)
        axes[i, j].set_title(f'{img_name} - {fusion_name}')
        axes[i, j].axis('off')

plt.tight_layout()
plt.show()

Output hidden; open in https://colab.research.google.com to view.